# Conversational Search

In this notebook you will implement the following steps:

- **Answer selection + evaluation**: Implement a *search-based* conversation framework evaluation framework to evaluate conversation topics made up of conversation turns.
- **Answer ranking**: Implement a *re-ranking method* to sort the initial search results. Evaluate the re-ranked results.
- **Conversation context**: Implement a conversational context modeling method to keep track of the conversation state. 

Submission dates:
- **20 October**: retrieval + evaluation
- **20 November**: pass
age re-ranking
- **20 December**: conversation state tracking

## Test bed and conversation topics
The TREC CAST corpus (http://www.treccast.ai/) for Conversational Search is indexed in this cluster and available to be searched behind an ElasticSearch API.

The queries and the relevance judgments are available through class `ConvSearchEvaluation`:

In [1]:
import TRECCASTeval as trec
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

import ElasticSearchSimpleAPI as es
import numpy as np

import pprint as pprint

test_bed = trec.ConvSearchEvaluation()

print()
print("========================================== Training conversations =====")
topics = {}
for topic in test_bed.train_topics:
    conv_id = topic['number']

    if conv_id not in (1, 2, 4, 7, 15, 17,18,22,23,24,25,27,30):
        continue

    print()
    print(conv_id, "  ", topic['title'])

    for turn in topic['turn']:
        turn_id = turn['number']
        utterance = turn['raw_utterance']
        topic_turn_id = '%d_%d'% (conv_id, turn_id)
        
        print(topic_turn_id, utterance)
        topics[topic_turn_id] = utterance

print()
print("========================================== Test conversations =====")
for topic in test_bed.test_topics:
    conv_id = topic['number']

    if conv_id not in (31, 32, 33, 34, 37, 40, 49, 50, 54, 56, 58, 59, 61, 67, 68, 69, 75, 77, 78, 79):
        continue

    print()
    print(conv_id, "  ", topic['title'])

    for turn in topic['turn']:
        turn_id = turn['number']
        utterance = turn['raw_utterance']
        topic_turn_id = '%d_%d'% (conv_id, turn_id)
        
        print(topic_turn_id, utterance)
        topics[topic_turn_id] = utterance



========================================== Training conversations =====

1    Career choice for Nursing and Physician's Assistant
1_1 What is a physician's assistant?
1_2 What are the educational requirements required to become one?
1_3 What does it cost?
1_4 What's the average starting salary in the UK?
1_5 What about in the US?
1_6 What school subjects are needed to become a registered nurse?
1_7 What is the PA average salary vs an RN?
1_8 What the difference between a PA and a nurse practitioner?
1_9 Do NPs or PAs make more?
1_10 Is a PA above a NP?
1_11 What is the fastest way to become a NP?
1_12 How much longer does it take to become a doctor after being an NP?

2    Goat breeds
2_1 What are the main breeds of goat?
2_2 Tell me about boer goats.
2_3 What breed is good for meat?
2_4 Are angora goats good for it?
2_5 What about boer goats?
2_6 What are pygmies used for?
2_7 What is the best for fiber production?
2_8 How long do Angora goats live?
2_9 Can you milk them?
2_10 How ma

Search example:

In [2]:
elastic = es.ESSimpleAPI()
results = elastic.search_body(topics['33_1'], numDocs = 10)
print(results)

    _index _type            _id     _score  \
0  msmarco  _doc  MARCO_6213095  13.568718   
1  msmarco  _doc  MARCO_5954177  12.347090   
2  msmarco  _doc  MARCO_6213093  12.184238   
3  msmarco  _doc  MARCO_5954180  12.141284   
4  msmarco  _doc  MARCO_6213094  11.893921   
5  msmarco  _doc  MARCO_7273401  11.884571   
6  msmarco  _doc  MARCO_1311833  11.654294   
7  msmarco  _doc  MARCO_6213091  11.639892   
8  msmarco  _doc  MARCO_7273397  11.605606   
9  msmarco  _doc  MARCO_3843635  11.429349   

                                        _source.body  
0  The film only adapts the first half of the boo...  
1  The NeverEnding Story (German: Die unendliche ...  
2  The NeverEnding Story was the first film adapt...  
3  The name of Atrayu's horse in the neverending ...  
4  Goodbye to the old Neverending Story and Hello...  
5  The NeverEnding Story III, starring Jason Jame...  
6  Noah Hathaway. Noah Leslie Hathaway (born Nove...  
7  The Neverending Story (German: Die unendliche ... 

/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


## Retrieval with the training conversations
The ElasticSearchSimpleAPI notebook illustrates how to use ElasticSearch. Use this API to retrieve the top 100 ranked passages for each conversation turn. 

To evaluate the results you should use the provided `ConvSearchEvaluation` class. Examine and discuss the recall metric results. In terms of metrics, discuss what should be your goals for each step of the project.

In [3]:
import TRECCASTeval as trec
import numpy as np

import ElasticSearchSimpleAPI as es
import numpy as np

import pprint as pprint

elastic = es.ESSimpleAPI()
test_bed = trec.ConvSearchEvaluation()

_recall = 0
_p10 = 0
_ndcg5 = 0

_nturns = 0

for topic in test_bed.train_topics:
    
    conv_id = topic['number']
    if conv_id not in (1, 2, 4, 7, 15, 17,18,22,23,24,25,27,30):
        continue
        
    for turn in topic['turn']:
        turn_id = turn['number']
        utterance = turn['raw_utterance']
        topic_turn_id = '%d_%d'% (conv_id, turn_id)
        
        aux = test_bed.relevance_judgments.loc[test_bed.relevance_judgments['topic_turn_id'] == (topic_turn_id)]
        num_rel = aux.loc[aux['rel'] != 0]['docid'].count()
        
        if num_rel == 0:
            continue

        result = elastic.search_body(query=utterance, numDocs = 100)

        if np.size(result) == 0 or num_rel == 0:
            print(topic_turn_id, utterance, num_rel, "NO RESULTS")
            continue
        else:
            print(topic_turn_id, utterance, num_rel)

        [p10, recall, ap, ndcg5] = test_bed.eval(result[['_id','_score']], topic_turn_id)

#        print('P10=', p10, '  Recall=', recall, '  NDCG=',ndcg)
        _recall = _recall + recall
        _p10 = _p10 + p10
        _ndcg5 = _ndcg5 + ndcg5
        
        _nturns = _nturns + 1


_p10 = _p10/_nturns
_recall = _recall/_nturns
_ndcg5 = _ndcg5/_nturns

print()
print('P10=', _p10, '  Recall=', _recall, '  NDCG@5', _ndcg5)


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


1_1 What is a physician's assistant? 5
1_2 What are the educational requirements required to become one? 11
1_3 What does it cost? 4


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


1_4 What's the average starting salary in the UK? 1
1_5 What about in the US? 12 NO RESULTS
1_6 What school subjects are needed to become a registered nurse? 8


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


...

/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


27_7 What happens if it's consumed in a smaller amount than recommended? 1
30_1 What are some advantages of using Linux? 3
30_2 Can I run Windows software in it? 7


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


30_3 How does it compare to Windows? 4
30_4 Which of these is more popular? 3
30_5 How do I install software on it? 3
30_6 What is an easy way to install Python? 5
30_7 Tell me about how I can share files. 6

P10= 0.11121495327102801   Recall= 0.3610449908347104   NDCG@5 0.14272400118644213


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


## Retrieval with the test conversations

In [4]:
import TRECCASTeval as trec
import numpy as np

from bert_serving.client import BertClient
from sklearn.metrics.pairwise import cosine_similarity

import ElasticSearchSimpleAPI as es
import numpy as np

import pprint as pprint

elastic = es.ESSimpleAPI()
test_bed = trec.ConvSearchEvaluation()

_recall = 0
_p10 = 0
_ndcg5 = 0
_nturns = 0
for topic in test_bed.test_topics:
    
    conv_id = topic['number']
    if conv_id not in (31, 32, 33, 34, 37, 40, 49, 50, 54, 56, 58, 59, 61, 67, 68, 69, 75, 77, 78, 79):
        continue

    for turn in topic['turn']:
        turn_id = turn['number']
        utterance = turn['raw_utterance']
        topic_turn_id = '%d_%d'% (conv_id, turn_id)
        
        aux = test_bed.test_relevance_judgments.loc[test_bed.test_relevance_judgments['topic_turn_id'] == (topic_turn_id)]
        num_rel = aux.loc[aux['rel'] != 0]['docid'].count()
        
        if num_rel == 0:
            continue

        result = elastic.search_body(query=utterance, numDocs = 100)

        if np.size(result) == 0 or num_rel == 0:
            print(topic_turn_id, utterance, num_rel, "NO RESULTS")
            continue
        else:
            print(topic_turn_id, utterance, num_rel)

        [p10, recall, ap, ndcg5] = test_bed.eval(result[['_id','_score']], topic_turn_id)

#        print('P10=', p10, '  Recall=', recall, '  NDCG=',ndcg)
        _recall = _recall + recall
        _p10 = _p10 + p10
        _ndcg5 = _ndcg5 + ndcg5
        
        _nturns = _nturns + 1

_p10 = _p10/_nturns
_recall = _recall/_nturns
_ndcg5 = _ndcg5/_nturns

print()
print('P10=', _p10, '  Recall=', _recall, '  NDCG@5', _ndcg5)


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


31_1 What is throat cancer? 89
31_2 Is it treatable? 77


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


31_3 Tell me about lung cancer. 171
31_4 What are its symptoms?  98


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


...

/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


79_5 How is his work related to Comte? 41
79_6 What is the functionalist theory? 77
79_7 What is its main criticism? 23


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


79_8 How does it compare to conflict theory? 51
79_9 What are modern examples of conflict theory? 40

P10= 0.23895348837209304   Recall= 0.16716739361136979   NDCG@5 0.18025607112012662


/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])
/home/ggoncalves/GoogleDrive/home/IdeaProjects/Project RI 2020/2020/ElasticSearchSimpleAPI.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(result["hits"]["hits"])


## Passage re-Ranking
The Passage Ranking notebook example illustrates how to use the BERT service to compute the similarity between sentences. Using the BERT service, improve a passage ranking method to rerank the initial retrieval step.

To evaluate the results you should use the provided `ConvSearchEvaluation` class.


## Conversation Context Modeling

Conversation State Tracking example ilustrates how to use the 

To evaluate the results you should use the provided `ConvSearchEvaluation` class.
